In [13]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import json
import baseline
from encoder import *
from baseline import *
from decoder import *
from alphabet import *
from train import *
import torch
from matplotlib import pyplot as plt

torch.__version__

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'1.4.0'

In [14]:
LANGUAGE = 'Python'

def filter_ascii(strings):
    'Returns only the strings that can be encoded in ASCII.'
    l = []
    for s in strings:
        try:
            s.encode('ascii')
            if len(s) <= 80:
                l.append(s)
        except UnicodeEncodeError:
            pass
        
    return l

with open('dataset/medium.json') as f:
    multilang_dataset = json.load(f)
    dataset = multilang_dataset[LANGUAGE]
    
    dataset['train'] = filter_ascii(dataset['train'])
    dataset['dev'] = filter_ascii(dataset['dev'])
    dataset['test'] = filter_ascii(dataset['test'])
    
    tiny_dataset = {
        'train': dataset['train'][:50],
        'dev': dataset['train'][:50],
        'test': dataset['train'][:50],
    }
    
    print('{} training examples, {} validation examples, {} test exampless'.format(
        len(dataset['train']), 
        len(dataset['dev']),
        len(dataset['test'])))

76738 training examples, 9590 validation examples, 9616 test exampless


In [15]:
def generate_dumb_dataset():
    'Returns a dataset where all examples are the same string, which consists of 10 times the same letter.'

    SIZE = 200
    l = []

    for i in range(SIZE):
        l.append(random.choice('abcdefghijklmnopqrstuvwxyz') * random.choice([5, 10]))
        
    return {'train': l, 'dev': l, 'test': l}

dumb_dataset = generate_dumb_dataset()

In [40]:
device = torch.device(0) if torch.cuda.is_available() else torch.device('cpu')
print(device)

alphabet = AsciiOneHotEncoding(device)
encoder = baseline.UniformEncoder(0.9)
decoder = AutoCompleteDecoderModel(alphabet, hidden_size=64)
nencoder = NeuralEncoder(alphabet, epsilon=0.5, hidden_size=64)

cpu
hello darkness


In [41]:
nencoder(["hello"])

tensor([[0.4914, 0.4903, 0.4899, 0.4910, 0.4916, 0.4899, 0.4911]],
       grad_fn=<SqueezeBackward1>)

In [42]:
print(nencoder.output_proj.weight._grad)


None


In [48]:
parameters = {
    'learning_rate': 5e-3,
    'verbose': True,
    'batch_size': 1,
    'init_scale': 0.01,
    'epochs': 3,
    'log_every':2
}

train_loss_history = train(nencoder, decoder, dumb_dataset, parameters, device)
plt.plot(train_loss_history)

batch:  ['xxxxx']
encoded_batch_probs:  tensor([[0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976]],
       grad_fn=<SqueezeBackward1>)
encoded_batch_strings: ['xxx']
per_prediction_loss:  tensor([29.1121], grad_fn=<SumBackward1>)
grad:  tensor([[-1.8264e-13,  2.7626e-13, -1.7167e-13,  ..., -3.6948e-13,
          1.6041e-13, -1.5306e-13],
        [ 8.3306e-14, -2.0668e-13,  5.3575e-14,  ...,  1.9181e-13,
         -1.1889e-13,  5.6185e-14],
        [ 1.3395e-13, -6.2295e-13, -2.9655e-15,  ...,  3.9274e-13,
         -3.5605e-13,  4.1427e-14],
        ...,
        [ 5.9640e-14, -3.2437e-14,  7.3775e-14,  ...,  1.0395e-13,
         -1.9621e-14,  5.9733e-14],
        [ 1.3061e-13, -1.8912e-13,  1.2536e-13,  ...,  2.6191e-13,
         -1.0997e-13,  1.1094e-13],
        [ 1.8296e-14,  1.0407e-13,  5.7596e-14,  ..., -1.3447e-15,
          5.8724e-14,  3.7449e-14]])
Epoch 0 iteration 0: loss = -4.248, tp = 27.27 lines/s, ETA 00h00m21s
batch:  ['yyyyy']
encoded_batch_probs:  tensor([[0.497

encoded_batch_probs:  tensor([[0.4976, 0.4976, 0.4977, 0.4977, 0.4977, 0.4977, 0.4976]],
       grad_fn=<SqueezeBackward1>)
encoded_batch_strings: ['ddd']
per_prediction_loss:  tensor([34.9195], grad_fn=<SumBackward1>)
grad:  tensor([[-1.0608e-05,  1.1103e-05,  1.0535e-05,  ..., -5.9990e-06,
          1.1745e-05,  8.0946e-06],
        [ 1.1136e-05, -1.1659e-05, -1.1061e-05,  ...,  6.3039e-06,
         -1.2327e-05, -8.4960e-06],
        [ 1.0836e-05, -1.1344e-05, -1.0763e-05,  ...,  6.1310e-06,
         -1.1997e-05, -8.2682e-06],
        ...,
        [ 1.1814e-05, -1.2347e-05, -1.1721e-05,  ...,  6.6557e-06,
         -1.3090e-05, -9.0202e-06],
        [ 1.0524e-05, -1.0999e-05, -1.0441e-05,  ...,  5.9297e-06,
         -1.1661e-05, -8.0352e-06],
        [-1.1400e-05,  1.1916e-05,  1.1311e-05,  ..., -6.4256e-06,
          1.2630e-05,  8.7033e-06]])
Epoch 0 iteration 10: loss = -6.025, tp = 24.55 lines/s, ETA 00h00m23s
batch:  ['eeeeeeeeee']
encoded_batch_probs:  tensor([[0.4976, 0.4976, 0

grad:  tensor([[ 9.2245e-33, -8.8695e-33, -8.4658e-33,  ..., -3.2483e-33,
          4.2021e-33,  3.3127e-33],
        [-9.2256e-33,  8.8706e-33,  8.4669e-33,  ...,  3.2487e-33,
         -4.2027e-33, -3.3132e-33],
        [-9.2247e-33,  8.8697e-33,  8.4661e-33,  ...,  3.2484e-33,
         -4.2022e-33, -3.3128e-33],
        ...,
        [-9.2258e-33,  8.8708e-33,  8.4671e-33,  ...,  3.2488e-33,
         -4.2027e-33, -3.3132e-33],
        [-9.2200e-33,  8.8652e-33,  8.4617e-33,  ...,  3.2467e-33,
         -4.2001e-33, -3.3111e-33],
        [ 9.2245e-33, -8.8695e-33, -8.4659e-33,  ..., -3.2483e-33,
          4.2021e-33,  3.3128e-33]])
batch:  ['lllll']
encoded_batch_probs:  tensor([[0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976]],
       grad_fn=<SqueezeBackward1>)
encoded_batch_strings: ['ll']
per_prediction_loss:  tensor([38.7027], grad_fn=<SumBackward1>)
grad:  tensor([[ 8.0104e-36, -7.8036e-36, -7.3605e-36,  ..., -2.7983e-36,
          3.5090e-36,  2.7237e-36],
        [-8.010

grad:  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
Epoch 0 iteration 32: loss = -18.179, tp = 26.83 lines/s, ETA 00h00m21s
batch:  ['gggggggggg']
encoded_batch_probs:  tensor([[0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976,
         0.4976, 0.4976, 0.4976]], grad_fn=<SqueezeBackward1>)
encoded_batch_strings: ['ggg']
per_prediction_loss:  tensor([69.9678], grad_fn=<SumBackward1>)
grad:  tensor([[-2.5173e-25,  2.5089e-25,  2.4648e-25,  ...,  5.1123e-26,
         -6.0787e-26, -4.9464e-26],
        [ 2.5173e-25, -2.5089e-25, -2.4648e-25,  ..., -5.1123e-26,
          6.0787e-26,  4.9464e-26],
        [ 2.5173e-25, -2.5089e-25, -2.4648e-25,  ..., -5.1123e-26,
          6.0787e-26,  4.9464e-26],
        ...,
        [ 2.5173e-25, -2.5089e-25, -2.4648e-25,  ..., 

grad:  tensor([[-6.6879e-26,  6.6780e-26,  6.5986e-26,  ...,  1.4828e-26,
         -1.7331e-26, -1.4515e-26],
        [ 6.6879e-26, -6.6780e-26, -6.5986e-26,  ..., -1.4828e-26,
          1.7331e-26,  1.4515e-26],
        [ 6.6879e-26, -6.6780e-26, -6.5986e-26,  ..., -1.4828e-26,
          1.7331e-26,  1.4515e-26],
        ...,
        [ 6.6879e-26, -6.6780e-26, -6.5986e-26,  ..., -1.4828e-26,
          1.7331e-26,  1.4515e-26],
        [ 6.6878e-26, -6.6779e-26, -6.5985e-26,  ..., -1.4827e-26,
          1.7331e-26,  1.4515e-26],
        [-6.6879e-26,  6.6780e-26,  6.5986e-26,  ...,  1.4828e-26,
         -1.7331e-26, -1.4515e-26]])
batch:  ['ttttt']
encoded_batch_probs:  tensor([[0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976]],
       grad_fn=<SqueezeBackward1>)
encoded_batch_strings: ['tt']
per_prediction_loss:  tensor([57.1271], grad_fn=<SumBackward1>)
grad:  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]

...

encoded_batch_strings: ['tttttt']
per_prediction_loss:  tensor([nan], grad_fn=<SumBackward1>)
grad:  tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
batch:  ['bbbbb']
encoded_batch_probs:  tensor([[0.4976, 0.4976, 0.4977, 0.4977, 0.4977, 0.4977, 0.4976]],
       grad_fn=<SqueezeBackward1>)
encoded_batch_strings: ['']
per_prediction_loss:  tensor([nan], grad_fn=<SumBackward1>)
grad:  tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
Epoch 1 iteration 67: loss = nan, tp = 25.02 lines/s, ETA 00h00m13s
batch:  ['dddd

grad:  tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
Epoch 1 iteration 83: loss = nan, tp = 24.77 lines/s, ETA 00h00m12s
batch:  ['mmmmmmmmmm']
encoded_batch_probs:  tensor([[0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976,
         0.4976, 0.4976, 0.4976]], grad_fn=<SqueezeBackward1>)
encoded_batch_strings: ['mmmm']
per_prediction_loss:  tensor([nan], grad_fn=<SumBackward1>)
grad:  tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
batch:  ['oooooooooo']
encoded_batch_probs:  tensor([[0.4

grad:  tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
Epoch 1 iteration 101: loss = nan, tp = 24.55 lines/s, ETA 00h00m12s
batch:  ['yyyyy']
encoded_batch_probs:  tensor([[0.4976, 0.4976, 0.4976, 0.4976, 0.4977, 0.4977, 0.4976]],
       grad_fn=<SqueezeBackward1>)
encoded_batch_strings: ['yy']
per_prediction_loss:  tensor([nan], grad_fn=<SumBackward1>)
grad:  tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
batch:  ['nnnnnnnnnn']
encoded_batch_probs:  tensor([[0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4

grad:  tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
batch:  ['lllll']
encoded_batch_probs:  tensor([[0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976, 0.4976]],
       grad_fn=<SqueezeBackward1>)
encoded_batch_strings: ['lll']
per_prediction_loss:  tensor([nan], grad_fn=<SumBackward1>)
grad:  tensor([[nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        ...,
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan],
        [nan, nan, nan,  ..., nan, nan, nan]])
Epoch 1 iteration 119: loss = nan, tp = 24.73 lines/s, ETA 00h00m11s
batch:  ['ddddd']
encoded_batch_probs:  tensor([[0.4976, 0.4976, 0.4977, 0.4977, 0.4977, 0.4977, 0.4976]

KeyboardInterrupt: 

In [123]:
ps = torch.tensor([0.3, 0.8], requires_grad=True)
bs = torch.bernoulli(ps)
s = bs.sum()
print(s)
s.backward()
print(bs)
print(bs.grad)

from torch.distributions import Categorical

b = Categorical(ps)
mask = b.sample()
# next_state, reward = env.step(action)
# loss = -m.log_prob(action) * reward
loss = b.log_prob(mask)
loss.backward()
print(loss)
print(mask.grad)

tensor(2., grad_fn=<SumBackward0>)
tensor([1., 1.], grad_fn=<BernoulliBackward0>)
None
tensor(-0.3185, grad_fn=<SqueezeBackward1>)
None


In [109]:
bs._grad

# End-to-end samples

In [47]:
SPLIT = 'train'

import copy

s = random.choice(dumb_dataset[SPLIT])
compressed = encoder.encode(s)
decompressed = decoder([compressed])

print('String:', repr(s))
print('Encoded:', repr(compressed))
print('Decoded:', repr(decompressed[0]))
print(len(decompressed))

String: 'ffffffffff'
Encoded: 'ffffffffff'
Decoded: 'd}{}\x13d}}|d}\x13\x13d})dddA}}\x13})}7d}}}|}}\x13d}\x13d}A|\x13}}Z(AA|\x13}}{}}}K}}AP}d}}\x13}d}}}}\x13d}j}}\x13Z\x13A}j}}}\x13dd})}\x13}Z\x13}|d\x13\x13}}}t})}\x13d}}dZ}}|}}d}Z}}dZAA}\x13})}dd}}dt}\x04d}dA}}}}A})d}}d}d}A}}\x04}A}A\x13d|A}}}}\x13A}\x13d}}}dA}}})d}d}{)\x13|A'
1


In [9]:
def top1accuracy(dataset):
    return len(list(filter(lambda s: s == decoder([encoder.encode(s)])[0],
                         dataset)))/len(dataset)
print(top1accuracy(dumb_dataset[SPLIT]))

0.25
